In [1]:
MAP_HEIGHT = 40
MAP_WIDTH = 40
WEIGHT_0 = 0.55
WEIGHTS = [WEIGHT_0, 1 - WEIGHT_0]
OPTIONS = [0, 1]
ITERATIONS = 5
# 0 - floor, 1 - wall
CHARS = {
    0: '\u25A2',
    1: '\u25A8'
}

STARVATION = 4
BIRTH = 3

In [2]:
def print_map(grid, file=None):
    print(file=file)
    for row in grid:
        row_str = ""
        for c in row:
            row_str += CHARS[c]
        print(row_str, file=file)

In [3]:
import random
from numpy import array, zeros_like, sum
def initialize_grid():
    grid = []
    for _ in range(MAP_WIDTH):
        row = [random.choices(OPTIONS, WEIGHTS)[0] for _ in range(MAP_HEIGHT)]
        grid.append(row)
    return array(grid)

In [4]:
grid = initialize_grid()
print_map(grid)


▨▨▨▢▨▨▨▢▨▢▨▨▢▨▨▢▢▢▢▨▢▢▢▢▢▨▨▢▢▨▢▨▢▨▨▨▨▢▨▢
▨▨▢▢▢▢▢▢▢▨▨▨▢▨▢▢▢▨▢▨▢▨▨▢▢▢▨▢▢▢▢▨▨▨▢▨▨▢▢▢
▢▨▢▢▢▢▨▨▨▢▢▨▨▨▢▢▢▨▨▨▢▨▢▢▢▢▨▢▨▢▨▢▨▢▢▨▢▢▨▢
▢▢▨▨▢▨▨▢▢▨▢▨▨▢▨▢▢▨▢▢▨▨▨▢▨▢▨▢▨▢▨▢▢▢▨▢▢▨▨▢
▢▢▨▨▢▢▢▨▨▨▢▢▢▨▢▢▨▨▢▢▢▢▢▢▢▢▢▨▢▨▢▢▢▨▢▢▨▢▨▢
▢▢▨▨▢▢▨▨▨▨▢▢▢▢▨▨▨▨▨▨▨▢▨▢▢▨▨▢▢▢▢▢▢▢▢▨▢▢▨▨
▢▢▨▢▨▨▨▨▢▨▢▨▢▢▨▢▨▢▨▢▢▨▨▨▢▨▨▨▢▢▨▨▨▨▢▢▨▨▢▢
▢▢▢▨▢▢▢▢▨▢▨▨▢▢▢▢▢▢▢▢▢▨▢▢▨▨▨▨▢▢▨▨▢▨▢▢▨▨▢▢
▢▨▨▢▢▨▢▢▨▢▢▢▢▨▢▨▢▢▨▨▨▨▢▢▨▢▨▢▢▨▨▨▢▢▢▢▢▨▢▢
▨▨▢▢▨▢▢▢▨▨▢▢▨▢▢▢▨▢▨▨▨▨▢▨▨▨▢▨▨▨▨▢▢▢▨▢▨▨▨▢
▨▢▢▢▨▨▢▢▨▢▨▢▢▢▢▢▨▨▨▢▢▨▨▨▨▨▢▨▢▨▢▢▢▨▢▢▨▨▨▨
▨▨▢▢▢▨▨▢▢▢▢▨▢▢▢▨▨▨▢▢▢▢▢▢▨▢▢▢▢▨▨▢▨▢▢▨▢▨▨▢
▢▨▨▢▢▨▨▨▢▢▢▢▢▢▨▨▨▨▨▢▢▢▨▨▢▨▢▢▢▢▢▢▢▨▢▢▢▢▨▢
▢▨▨▨▢▢▢▢▢▢▢▢▨▨▨▨▢▨▢▨▢▢▨▢▢▨▨▢▨▨▨▨▨▢▢▨▢▨▨▢
▨▢▢▨▢▢▨▢▨▢▢▢▨▢▢▨▨▢▨▨▢▨▢▨▨▢▢▨▢▢▨▢▢▢▢▨▨▨▢▨
▢▢▢▢▢▢▢▢▢▢▢▨▨▢▢▢▨▢▢▨▢▢▨▨▨▨▢▢▢▢▢▢▨▨▢▢▢▢▨▢
▨▨▢▢▢▢▨▢▨▨▨▢▨▢▢▨▢▨▢▢▢▢▢▨▨▢▨▨▢▢▢▢▢▨▢▨▨▨▢▢
▨▢▨▢▨▢▨▨▨▢▢▨▨▢▨▢▢▢▨▢▢▢▢▢▢▨▨▨▢▨▢▨▨▨▢▢▨▨▨▢
▨▢▢▨▨▨▢▨▢▢▢▨▨▢▢▢▢▨▢▨▢▢▢▢▢▨▢▢▢▨▢▢▢▢▨▢▢▨▢▨
▨▢▢▢▢▢▢▢▨▢▢▨▢▨▨▢▢▨▨▨▢▨▨▨▨▨▨▢▨▢▨▢▢▢▨▢▢▨▨▨
▢▢▨▢▨▢▨▢▢▢▨▢▢▨▢▢▨▨▨▢▢▨▢▨▨▢▢▢▢▢▢▨▨▨▢▢▨▢▨▢
▢▨▢▨▨▨▨▢▢▢▨▨▨▢▨▢▢▢▨▨▨▨▨▢▢▢▢▢▨▢▢▨▢▢▢▨▨▢▢▢
▢▨▨▨▨▢▢▢▨▨▢▨▢▢▢▢▢▨▢▨▢▢▢▨▢▨▢▢▨▨▢▢▨▨▨▢▢▢▢▢
▢▢▢▨▢▢▨▨▢▢▨▢▨▨▢▢▨▢▢▨▢▨▨▨▨▢▢▨▨▨▨▢▢▢▢▢▨▢▢▨
▨▢▢▢▨▢▨▨▢▨▢▢▢▢▨

In [5]:
def count_alive_neighbors(board, x, y, n):
    """Sums the value of a 2D array's slice n-steps in every direction from P(x,y)"""
    try:
        value = board[y][x]
    except IndexError:
        return None
    board_width = len(board[0])
    board_height = len(board)
    board_slice = [row[max(x-n, 0):min(x+n+1, board_width)] for row in board[max(0, y-n):min(y+n+1, board_height)]]
    return sum(board_slice) - value

In [6]:
def calculate_new_state(old_value, alive_neighbors_1, alive_neighbors_2):
    if old_value:
        if alive_neighbors_1 <= STARVATION:
            return 0
        else:
            return 1
    else:
        if alive_neighbors_2 <= BIRTH:
            return 1
        else:
            return 0

def do_step(board):
    new_board = zeros_like(board)
    for y, row in enumerate(new_board):
        for x, _ in enumerate(row):
            if x == 0 or x == MAP_WIDTH-1 or y == 0 or y == MAP_HEIGHT -1:
                new_board[y][x] = 1
                continue
            old_element = board[y][x]
            alive_neighbors_1 = count_alive_neighbors(board, x, y, 1)
            alive_neighbors_2 = count_alive_neighbors(board, x, y, 2)
            new_board[y][x] = calculate_new_state(old_element, alive_neighbors_1, alive_neighbors_2)    
    return new_board

In [7]:
POSSIBILITIES = list(range(9))

with open("grids_settings_testing.txt", "w", encoding='utf8') as f:
    for death in POSSIBILITIES:
        for birth in POSSIBILITIES:
            STARVATION = death
            BIRTH = birth
            random.seed(123)
            grid = initialize_grid()
            for _ in range(ITERATIONS):
                grid = do_step(grid)
            print(f"STARVATION IF ABOVE: {STARVATION}, BIRTH AT {BIRTH}", file=f)
            print_map(grid, f)

In [8]:
STARVATION = 4
BIRTH = 1
ITERATIONS = 3

with open(f"grids_{STARVATION}_{BIRTH}_{ITERATIONS}_testing.txt", "w", encoding='utf8') as f:
    for sd in list(range(50)):
        random.seed(sd)
        grid = initialize_grid()
        for _ in range(ITERATIONS):
            grid = do_step(grid)
        print(f"SEED {sd}", file=f)
        print_map(grid, f)

In [9]:
import json
with open("mockgrid.json", "w") as j:
    json.dump(grid.tolist(), j)

In [13]:
MAP_WIDTH = 80
MAP_HEIGHT = 80

STARVATION = 4
BIRTH = 1
ITERATIONS = 3

random.seed(123)

grid_for_processing = initialize_grid()
for _ in range(ITERATIONS):
    grid_for_processing = do_step(grid_for_processing)
print_map(grid_for_processing)


▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨
▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨
▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨
▨▢▢▢▨▨▢▢▢▨▢▢▢▨▨▨▨▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▢▢▢▨▨▨▨▢▢▢▢▢▨▢▢▢▢▢▢▢▢▢▢▨▢▨▨▨▨▨▢▢▢▢▨▨▢▢▢▢▢▢▨▢▢▨
▨▢▢▢▨▨▢▢▢▨▢▢▨▨▨▨▨▨▢▢▢▢▨▨▨▨▨▨▨▨▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▢▢▢▢▨▨▢▢▨▨▢▢▢▢▢▨
▨▢▢▢▢▢▢▢▢▨▢▢▢▨▨▨▨▨▢▢▢▢▢▨▨▨▨▨▨▨▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▢▢▢▢▨▨▢▢▨▨▨▢▢▢▢▨
▨▢▢▢▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▢▢▢▨▨▨▨▨▨▨▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▢▢▢▨▨▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▢▢▢▨▨▨▨▢▢▢▨
▨▢▢▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▢▢▨▨▨▨▨▨▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▢▢▨▨▨▨▨▨▢▢▨
▨▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▢▢▨▨▨▨▨▨▢▢▨
▨▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▨▨▨▨▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▢▢▢▨▨▨▨▨▢▢▢▨
▨▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▨▨▨▨▨▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▨▨▢▢▢▢▢▨
▨▢▢▢▢▢▢▢▨▨▢▢▢▢▢▢▨▨▨▨▨▨▨▨▢▢▨▨▨▨▨▨▨▨▢▢▢▨▨▨▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▢▨▨▨▨▨▨▨▨▨▢▢▢▢▢▢▢▢▢▢▢▢▢▨
▨▢▢▨▨▢▢▢▨▨▢▢▢▢▢▢▢▢▨▨▨▨▨▢▢▨▨

In [11]:
import json
with open("mockgrid_big.json", "w") as j:
    json.dump(grid.tolist(), j)